In [1]:
import h5py
import sys
sys.path.append('/Users/zhengj10/Python/Modules')
import os
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
import yaml
import pandas as pd
import PatcherBot as pb
from glob import glob
import seaborn as sns


In [2]:
columns = ['Date', 'Plate', 'Well', 'Cell', 'Mutation', 'Transfected_date', 'Leak',
       'Ra', 'dFF', 'F0', 'amp_on_single', 'amp_off_single', 'tau_on_single',
       'tau_off_single', 'bleach_x', 'bleach', 'amp_on_f_double',
       'amp_on_s_double', 'amp_off_f_double', 'amp_off_s_double',
       'tau_on_f_double', 'tau_off_f_double', 'tau_on_s_double',
       'tau_off_s_double', 'time_on_10-90', 'time_off_10-90', 'Leak_min',
       'Ra_max', 'Objective', 'peak_dFF']

df = pd.DataFrame(columns = columns)
df


,Date,Plate,Well,Cell,Mutation,Transfected_date,Leak,Ra,dFF,F0,...,tau_on_f_double,tau_off_f_double,tau_on_s_double,tau_off_s_double,time_on_10-90,time_off_10-90,Leak_min,Ra_max,Objective,peak_dFF


### Load params from .mat file

In [3]:
# sumPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/summary/Voltron2_paper'
imgPath = '/Volumes/genie/BenArthur/patch-screen/bot'
rawPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw'

# sumPath = '/Users/zhengj10/Desktop/VC_reprocess_all'
sumPath = '/Users/zhengj10/Desktop/GENIE/Data/PatcherBot/summary'
# imgPath = '/Volumes/Backup Plus/patch-screen/bot'
# rawPath = '/Volumes/Backup Plus/PatcherBot/raw'

os.path.exists(rawPath)


True

In [108]:
# datePath = pb.getPathInfo(imgPath)
datePath = ['2021.02.24_000']
datePath

['2021.02.24_000']

#### Append all parameters except Objective and peak_dFF

In [109]:
data_all={}
for param in(df.columns):
    data_all[param] = []
for date in datePath:
    platePath = pb.getPathInfo(os.path.join(imgPath, date))
    
    for plate in platePath:
        
        print(os.path.join(rawPath, date, plate))

        idxFile = os.path.join(rawPath, date, plate, '.index')
        a = yaml.load(open(idxFile), Loader=yaml.FullLoader)
        try:
            mutation = a['.']['mutation_1']
        except:
            mutation = a['.']['mutation_2']
        try:
            note = a['.']['notes'][0:6]
        except:
            note = 'Not recorded'
        cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
        for cell in(cellPath):

            
            matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
            if len(matFiles) != 9:
                continue


            data_all['Date'].append(date)
            data_all['Plate'].append(plate)
            data_all['Well'].append('well_1')
            data_all['Cell'].append(cell)
            data_all['Mutation'].append(mutation)
            data_all['Transfected_date'].append(note[:6])
            leaks = []
            Ra_s = [] # List of Ra
            F0 = []

            dff=[]
            amp_on = []
            amp_off = []
            tau_on = []
            tau_off = []

            amp_on_fast = []
            amp_on_slow = []
            amp_off_fast = []
            amp_off_slow = []
            tau_on_fast = []
            tau_on_slow = []
            tau_off_fast = []
            tau_off_slow = []
            time_on = []
            time_off = []
            bleach_x = []
            bleach = []



            for i in range(9):

                ivFile = os.path.join(rawPath, date, plate, cell) + '/camera_clamp1_000/' + str(i).zfill(3) + '/Clamp1.ma'
                
                if not os.path.exists(ivFile):
                    ivFile = os.path.join(rawPath, date, plate, cell) + '/camera_clamp2_000/' + str(i).zfill(3) + '/Clamp2.ma'
                

                leak = pb.get_leak(ivFile)
                Ra = pb.get_Ra(ivFile,i)




                matFile = os.path.join(imgPath, date, plate, cell) + '/' + str(i).zfill(3) + '.mat'
#                 print(matFile)
                data = pb.get_data(matFile)
                f0_single = data['f0']

                leaks.append(leak)
                Ra_s.append(Ra)
                F0.append(f0_single)


                trace_single = data['dff_corrected_single']               # for single fit
                dff_single = pb.get_dff(trace_single)
                amp_on_single = data['amp_on'][0]
                amp_off_single = data['amp_off'][0]
                tau_on_single = data['tau_on'][0]
                tau_off_single = data['tau_off'][0]
                # bleach_x1 = data['bleach_x'][0]
                # bleach_x2 = data['bleach_x'][1]
                
                bleach_x1 = 0
                bleach_x2 = 0
                
                                
                bleach_single_1 = 0
                bleach_single_2 = 0
                
                # bleach_single_1 = data['bleach'][0]
                # bleach_single_2 = data['bleach'][1]



                dff.append(dff_single)
                amp_on.append(amp_on_single)
                amp_off.append(amp_off_single)
                tau_on.append(tau_on_single)
                tau_off.append(tau_off_single)
                bleach_x.append(bleach_x1)
                bleach_x.append(bleach_x2)
                bleach.append(bleach_single_1)
                bleach.append(bleach_single_2)


                trace_double = data['dff_corrected_double']                # for double fit
                amp_on_fast_single = data['amp_on_fast'][0]                
                amp_on_slow_single = data['amp_on_slow'][0]
                amp_off_fast_single = data['amp_off_fast'][0]
                amp_off_slow_single = data['amp_off_slow'][0]
                tau_on_fast_single = data['tau_on_fast'][0]
                tau_on_slow_single = data['tau_on_slow'][0]
                tau_off_fast_single = data['tau_off_fast'][0]
                tau_off_slow_single = data['tau_off_slow'][0]
                time_on_double = data['time_on'][0]
                time_off_double = data['time_off'][0]
                amp_on_fast.append(amp_on_fast_single)
                amp_on_slow.append(amp_on_slow_single)
                amp_off_fast.append(amp_off_fast_single)
                amp_off_slow.append(amp_off_slow_single)
                tau_on_fast.append(tau_on_fast_single)
                tau_on_slow.append(tau_on_slow_single)
                tau_off_fast.append(tau_off_fast_single)
                tau_off_slow.append(tau_off_slow_single)
                time_on.append(time_on_double)
                time_off.append(time_off_double)

            data_all['F0'].append(F0)
            data_all['Leak'].append(leaks)
            data_all['Ra'].append(Ra_s)
            data_all['Leak_min'].append(np.min(leaks))
            data_all['Ra_max'].append(np.max(Ra_s))


            data_all['dFF'].append(dff)
            data_all['amp_on_single'].append(amp_on)
            data_all['amp_off_single'].append(amp_off)
            data_all['tau_on_single'].append(tau_on)
            data_all['tau_off_single'].append(tau_off)
            data_all['time_on_10-90'].append(time_on)
            data_all['time_off_10-90'].append(time_off)
            data_all['bleach_x'].append(bleach_x)
            data_all['bleach'].append(bleach)

            data_all['amp_on_f_double'].append(amp_on_fast)
            data_all['amp_on_s_double'].append(amp_on_slow)
            data_all['amp_off_f_double'].append(amp_off_fast)
            data_all['amp_off_s_double'].append(amp_off_slow)
            data_all['tau_on_f_double'].append(tau_on_fast)
            data_all['tau_on_s_double'].append(tau_on_slow)
            data_all['tau_off_f_double'].append(tau_off_fast)
            data_all['tau_off_s_double'].append(tau_off_slow)

        
print('Done!')


/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.02.24_000/plate_000
Done!


In [110]:
for k,v in data_all.items():
    print(k,len(v))

Date 8
Plate 8
Well 8
Cell 8
Mutation 8
Transfected_date 8
Leak 8
Ra 8
dFF 8
F0 8
amp_on_single 8
amp_off_single 8
tau_on_single 8
tau_off_single 8
bleach_x 8
bleach 8
amp_on_f_double 8
amp_on_s_double 8
amp_off_f_double 8
amp_off_s_double 8
tau_on_f_double 8
tau_off_f_double 8
tau_on_s_double 8
tau_off_s_double 8
time_on_10-90 8
time_off_10-90 8
Leak_min 8
Ra_max 8
peak_dFF 0
Objective 0


In [111]:
del data_all['peak_dFF']
del data_all['Objective']


In [112]:
df_new = pd.DataFrame(data_all)


In [113]:
df_new['peak_dFF'] = 'N/A'
df_new['Objective'] = '40x oil GENIE'


In [101]:
df = df[:-6]

In [114]:
df = df.append(df_new,ignore_index=True)
df.index = np.arange(len(df))

In [115]:
len(df)

789

In [106]:
df

,Date,Plate,Well,Cell,Mutation,Transfected_date,Leak,Ra,dFF,F0,...,tau_on_f_double,tau_off_f_double,tau_on_s_double,tau_off_s_double,time_on_10-90,time_off_10-90,Leak_min,Ra_max,peak_dFF,Objective
0,2020.01.16_000,plate_000,well_1,patch_attempt_0000_000,421.4919,Not re,"[-273.30791454288993, -247.2847961937989, -230...","[12.562946620032447, 11.672313636531248, 11.42...","[0.1240664966930221, 0.03287559321130864, -0.0...","[55.95911727215224, 50.114173041791965, 49.453...",...,"[1.6291896590311268, 0.08343922958853289, 1.40...","[3.3658439575825523, 0.00731864723109076, 30.0...","[5.553002784589947, 0.27641276530846143, 1.426...","[648.150726219816, 939.9844617280704, 0.351552...","[5.827952075130611, nan, nan, 2.81215194138371...","[13.586295888033192, 0.01609724338823071, 60.7...",-273.307915,12.562947,N/A,40x air
1,2020.01.16_000,plate_000,well_1,patch_attempt_0001_000,421.4919,Not re,"[-237.3972807384172, -218.15358381535427, -203...","[11.95412054090735, 12.812385279643598, 11.398...","[0.13382765998673282, 0.04390390951978096, -0....","[56.18196486351705, 53.505740488511385, 51.916...",...,"[1.8254069115378746, 1.217044208362325, 1.8518...","[4.858194793742291, 0.029944907915363923, 2.89...","[14.347918384703465, 10.390194443666934, 17.12...","[546.184186180203, 32.125764248158816, 1837.80...","[13.381707271911466, 3.453745880552276, 23.698...","[13.248265048029907, 8.562445469229942, 8.0710...",-237.397281,14.573183,N/A,40x air
2,2020.01.16_000,plate_000,well_1,patch_attempt_0003_000,421.4919,Not re,"[-355.04183136603655, -433.6584719269079, -367...","[16.792550469234243, 13.926753101639353, 16.59...","[0.17148085069629138, 0.038436821375376, -0.03...","[38.57227164999943, 41.81442493532453, 33.3212...",...,"[8.806177607703148, 9.18006619337648, 0.124973...","[0.05261023191305472, 14.90536112610127, 0.162...","[184.45152694768322, 100.06500502828118, 784.1...","[494.19257544524106, 16.1065241705335, 164.169...","[162.47644355584657, 91.02766623883844, 498.44...","[296.37435070762035, 33.23406602386058, 131.45...",-1124.129558,16.792550,N/A,40x air
3,2020.01.16_000,plate_000,well_1,patch_attempt_0004_000,421.4919,Not re,"[-105.68897765894373, -95.17894141787997, -90....","[25.682033952870928, 20.663532332561235, 23.47...","[0.13659093129304908, 0.03461940243069092, -0....","[50.41897821259977, 48.095856589923, 46.541388...",...,"[0.433968004977366, 0.027791428458894592, 16.1...","[4.146286825290511, 13.502274963126936, 0.3658...","[136.2587668824007, 630947.7293965054, 1914.72...","[675.7236549940974, 13.9403411436837, 2165.874...","[1.6237786798697016, 0.06105804664412062, 536....","[17.082498692505396, 29.806340772112435, 1.445...",-105.688978,26.875255,N/A,40x air
4,2020.01.16_000,plate_000,well_1,patch_attempt_0006_000,421.4919,Not re,"[-193.34763603632365, -184.78394445852737, -17...","[8.014390783496028, 8.11197701432406, 7.772876...","[0.1372869034783221, 0.03690337347208496, -0.0...","[56.778489013250166, 53.791543250936115, 53.17...",...,"[3.7416485790803558, 0.195587180948756, 0.4307...","[2.803105129112075, 4.269587628702164, 2.02954...","[190.640149351619, 2.940677979841209, 992.7884...","[373.904320505812, 0.013588388112271208, 601.1...","[15.912284659557542, 2.4808133389355476, 446.0...","[12.46828206225814, 9.241195979013582, 198.681...",-193.347636,12.126622,N/A,40x air
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,2020.11.25_000-varnam,plate_000,well_1,patch_attempt_0005_000,557.1,201018,"[-10.24237005098188, -11.435132934459052, -11....","[25.9435192173165, 10.011422898950288, 9.85716...","[0.01514997276649705, 0.005752500944513999, -0...","[88.36868137300952, 87.36886793967865, 87.2751...",...,"[1.655015703608406, 0.7999645359597429, 19.139...","[2.349434791923179, 149.06283276737494, 9.2482...","[93.72263900133734, 374.87281530172055, 18.966...","[2350.072879806937, 531.4822656368768, 16976.5...","[74.15964902857746, 565.5574473024757, 42.1069...","[5.1168

#### Append peak_dFF

In [ ]:
# datePath = [
# '2020.08.20_000-voltron',
# '2020.08.26_000-voltron',
# '2020.08.27_000-voltron'
# ]

In [95]:
data_all={}
data_all['Date'] = []
data_all['Plate'] = []
data_all['Well'] = []
data_all['Cell'] = []
data_all['Mutation'] = []
data_all['peak_dFF'] = []

for date in datePath:
    platePath = pb.getPathInfo(os.path.join(imgPath, date))
#     print(date)
    
    for plate in platePath:
#         print(os.path.join(rawPath, date, plate))

        idxFile = os.path.join(rawPath, date, plate, '.index')
        a = yaml.load(open(idxFile), Loader=yaml.FullLoader)
        try:
            mutation = a['.']['mutation_1']
        except:
            mutation = a['.']['mutation_2']
        try:
            note = a['.']['notes'][0:6]
        except:
            note = 'Not recorded'
        cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
        for cell in(cellPath):
            matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
            if len(matFiles) != 9:
                continue


            data_all['Date'].append(date)
            data_all['Plate'].append(plate)
            data_all['Well'].append('well_1')
            data_all['Cell'].append(cell)
            data_all['Mutation'].append(mutation)
            

#             if date in ['2020.08.20_000-voltron','2020.08.26_000-voltron','2020.08.27_000-voltron']:
#                 dFF_p = []
#                 matFiles = glob(os.path.join(imgPath,date,plate,cell)+'/*.mat')
#                 for matFile in matFiles:
#                     f = h5py.File(matFile,'r')
#                     d = f['params']['fit_pix_alpha_single']
#                     st = d[0]
#                     obj = f[st]
#                     dff_p = obj['bkg_norm_ave_dFoF0_transient'][()][0]
#                     dFF_p.append(dff_p)
#                 data_all['peak_dFF'].append(dFF_p)
#             else:
#                 data_all['peak_dFF'].append('N/A')


            


            print(os.path.join(imgPath,date,plate,cell))

            dFF_p = []
            matFiles = glob(os.path.join(imgPath,date,plate,cell)+'/*.mat')
            matFiles = sorted(matFiles)

            if len(matFiles)<9:
                continue

            for matFile in matFiles:
                f = h5py.File(matFile,'r')
                d = f['params']['fit_pix_alpha_single']
                st = d[0]
                obj = f[st]
                dff_p = obj['bkg_norm_ave_dFoF0_transient'][()][0]
                dFF_p.append(dff_p)
            data_all['peak_dFF'].append(dFF_p)

            

print('Done!')


/Volumes/genie/BenArthur/patch-screen/bot/2020.11.25_000-varnam/plate_000/patch_attempt_0003_000
/Volumes/genie/BenArthur/patch-screen/bot/2020.11.25_000-varnam/plate_000/patch_attempt_0005_000
/Volumes/genie/BenArthur/patch-screen/bot/2020.11.25_000-varnam/plate_000/patch_attempt_0006_000
/Volumes/genie/BenArthur/patch-screen/bot/2020.11.25_000-varnam/plate_000/patch_attempt_0007_000
/Volumes/genie/BenArthur/patch-screen/bot/2020.11.25_000-varnam/plate_000/patch_attempt_0008_000
/Volumes/genie/BenArthur/patch-screen/bot/2020.11.25_000-varnam/plate_000/patch_attempt_0009_000
Done!


#### Append Objective

In [63]:
df = pd.read_pickle(sumPath+'/Summary_VC_all.pkl')

In [69]:
df['Objective']='40x air'

In [6]:
objectives = []
for i,r in df.iterrows():
#     print(i)
    idxFile = os.path.join(rawPath, r['Date'], r['Plate'], '.index')
#     idxFile = os.path.join(rawPath, r['Date'][:14], r['Plate'], '.index')
    a = yaml.load(open(idxFile), Loader=yaml.FullLoader)
    try:
        mutation = a['.']['mutation_1']
    except:
        mutation = a['.']['mutation_2']
    try:
        note = a['.']['notes'][0:6]
    except:
        note = 'Not recorded'
    if not 'notes' in a['.'].keys():
        objectives.append('40x air')
        continue
    elif 'GENIE' in(a['.']['notes']):
        objectives.append('40x oil GENIE')
    elif 'core' in (a['.']['notes']):
        objectives.append('40x oil core')
    else:
        objectives.append('40x air')


In [87]:
df.index = np.arange(len(df))

In [7]:
df['Objective']=objectives

In [8]:
df

,Date,Plate,Well,Cell,Mutation,Transfected_date,Leak,Ra,dFF,F0,...,tau_on_f_double,tau_off_f_double,tau_on_s_double,tau_off_s_double,time_on_10-90,time_off_10-90,Leak_min,Ra_max,peak_dFF,Objective
0,2020.01.16_000,plate_000,well_1,patch_attempt_0000_000,421.4919,Not re,"[-273.30791454288993, -247.2847961937989, -230...","[12.562946620032447, 11.672313636531248, 11.42...","[0.1240664966930221, 0.03287559321130864, -0.0...","[55.95911727215224, 50.114173041791965, 49.453...",...,"[1.6291896590311268, 0.08343922958853289, 1.40...","[3.3658439575825523, 0.00731864723109076, 30.0...","[5.553002784589947, 0.27641276530846143, 1.426...","[648.150726219816, 939.9844617280704, 0.351552...","[5.827952075130611, nan, nan, 2.81215194138371...","[13.586295888033192, 0.01609724338823071, 60.7...",-273.307915,12.562947,N/A,40x air
1,2020.01.16_000,plate_000,well_1,patch_attempt_0001_000,421.4919,Not re,"[-237.3972807384172, -218.15358381535427, -203...","[11.95412054090735, 12.812385279643598, 11.398...","[0.13382765998673282, 0.04390390951978096, -0....","[56.18196486351705, 53.505740488511385, 51.916...",...,"[1.8254069115378746, 1.217044208362325, 1.8518...","[4.858194793742291, 0.029944907915363923, 2.89...","[14.347918384703465, 10.390194443666934, 17.12...","[546.184186180203, 32.125764248158816, 1837.80...","[13.381707271911466, 3.453745880552276, 23.698...","[13.248265048029907, 8.562445469229942, 8.0710...",-237.397281,14.573183,N/A,40x air
2,2020.01.16_000,plate_000,well_1,patch_attempt_0003_000,421.4919,Not re,"[-355.04183136603655, -433.6584719269079, -367...","[16.792550469234243, 13.926753101639353, 16.59...","[0.17148085069629138, 0.038436821375376, -0.03...","[38.57227164999943, 41.81442493532453, 33.3212...",...,"[8.806177607703148, 9.18006619337648, 0.124973...","[0.05261023191305472, 14.90536112610127, 0.162...","[184.45152694768322, 100.06500502828118, 784.1...","[494.19257544524106, 16.1065241705335, 164.169...","[162.47644355584657, 91.02766623883844, 498.44...","[296.37435070762035, 33.23406602386058, 131.45...",-1124.129558,16.792550,N/A,40x air
3,2020.01.16_000,plate_000,well_1,patch_attempt_0004_000,421.4919,Not re,"[-105.68897765894373, -95.17894141787997, -90....","[25.682033952870928, 20.663532332561235, 23.47...","[0.13659093129304908, 0.03461940243069092, -0....","[50.41897821259977, 48.095856589923, 46.541388...",...,"[0.433968004977366, 0.027791428458894592, 16.1...","[4.146286825290511, 13.502274963126936, 0.3658...","[136.2587668824007, 630947.7293965054, 1914.72...","[675.7236549940974, 13.9403411436837, 2165.874...","[1.6237786798697016, 0.06105804664412062, 536....","[17.082498692505396, 29.806340772112435, 1.445...",-105.688978,26.875255,N/A,40x air
4,2020.01.16_000,plate_000,well_1,patch_attempt_0006_000,421.4919,Not re,"[-193.34763603632365, -184.78394445852737, -17...","[8.014390783496028, 8.11197701432406, 7.772876...","[0.1372869034783221, 0.03690337347208496, -0.0...","[56.778489013250166, 53.791543250936115, 53.17...",...,"[3.7416485790803558, 0.195587180948756, 0.4307...","[2.803105129112075, 4.269587628702164, 2.02954...","[190.640149351619, 2.940677979841209, 992.7884...","[373.904320505812, 0.013588388112271208, 601.1...","[15.912284659557542, 2.4808133389355476, 446.0...","[12.46828206225814, 9.241195979013582, 198.681...",-193.347636,12.126622,N/A,40x air
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,2020.10.28_001-varnam,plate_002,well_1,patch_attempt_0032_000,558.1,201020,"[-25.20637164794756, -30.012874129151477, -44....","[7.985028475019366, 7.818388577771687, 7.55842...","[0.004208819231560889, 0.002447035456243152, 0...","[245.26024421012883, 239.77437673247695, 235.8...",...,"[0.0746472168924274, 27.47551395192625, 35.724...","[186.19253271637544, 246.80337306763678, 56.34...","[129.8177892753783, 498.7222455523758, 208.327...","[233.3133346673759, 446.08794253403624, 5753.6...","[257.68297786440854, 376.39853213750723, 53.44...","[127.64

In [64]:
# df_new = pd.DataFrame(data_all)
# df_new
for k,v in data_all.items():
    print(k,len(v))
# del data_all['peak_dFF']

Date 673
Plate 673
Well 673
Cell 673
Mutation 673
peak_dFF 673


In [ ]:
# for k,v in data_all.items():
#     print(k,len(v))

In [55]:
# # df_new_1 = pd.DataFrame(data_all, columns = list(df)) # columns specification keeps the order of columns; index specification makes sure dict keys are the columns 
df = df.append(df_new, ignore_index = True)
# df.index = np.arange(len(df))
df.shape


(673, 29)

In [116]:
df.to_pickle(sumPath+'/Summary_VC_all.pkl')
df.to_excel(sumPath+'/Summary_VC_all.xlsx')
# # df.to_excel(sumPath + '/summary.xlsx')